Generate map data to be used on a custom Google Map https://www.google.com/mymaps


In [ ]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
print("Imported libs")

In [ ]:
colocs = pl.read_csv("seen_colocs.csv")
colocs

In [ ]:
world_cities = pl.read_csv("worldcities.csv", ignore_errors=True)
world_cities

In [ ]:
data = pl.read_csv("full_scan.csv")
data = data.with_columns(
    pl.from_epoch("timestamp", time_unit="s")
)
data

In [ ]:
colocs_with_coords = colocs.join(world_cities, left_on="city", right_on="city_ascii", how="left")
colocs_with_coords = colocs_with_coords.select(["colocationCenter", "city", "country","lat", "lng"]).unique(subset=["colocationCenter"])
colocs_with_coords

In [ ]:
data_grouped_by_colocation = (
    data
    .group_by("balancerColocationCenter")
    .agg(
        pl.col("balancerId").unique().count().alias("balancerCount")
    )
)
with_coords = data_grouped_by_colocation.join(colocs_with_coords, left_on="balancerColocationCenter", right_on="colocationCenter", how="left")
with_coords = with_coords.select(["balancerColocationCenter", "city", "lat", "lng", "balancerCount"])
with_coords
